In [1]:
import numpy as np
import pandas as pd
from edge_detect import normalize, detect_lines
import torch
import os

In [2]:
data_dir = "DAS_data"
files = os.listdir(data_dir)
print(len(files))

2334


In [3]:
df = pd.read_excel("das_gopro_mapping_conf0.4.xlsx")
values = ['DAS_File', 'DAS_DateTime', 'DAS_Start_Seconds', 'DAS_End_Seconds',
       'GoPro_Start_Video', 'GoPro_Start_Time', 'GoPro_Start_Seconds',
       'GoPro_Start_Scaled_Time', 'GoPro_Start_Scaled_Seconds',
       'GoPro_End_Video', 'GoPro_End_Time', 'GoPro_End_Seconds',
       'GoPro_End_Scaled_Time', 'GoPro_End_Scaled_Seconds', 'class_0_count',
       'class_0_tracks', 'class_1_count', 'class_1_tracks', 'class_2_count',
       'class_2_tracks', 'class_3_count', 'class_3_tracks', 'class_5_count',
       'class_5_tracks', 'class_6_count', 'class_6_tracks', 'class_7_count',
       'class_7_tracks']
fill = {}
for c in values:
    if "count" in c:
        fill[c] = 0
    elif "tracks" in c:
        fill[c] = "[]"
    else:
        fill[c] = ""
df = df.fillna(value = fill)

In [4]:
def das_to_numpy(f):
    new_name = "DAS_data/FBE_sensor"
    f = f.replace("sensor", "")
    f = f.replace(".png", "")
    f = f.replace(".h5", "")
    f = f.replace("Start", "")
    f = f.replace("End", "")
    f = f.replace("0.0_30.0", "0s-30s")
    f = f.replace("30.0_60.0", "30s-60s")
    f = new_name + f + "_10-200Hz.npy"
    return f

In [22]:
from tqdm import tqdm
imgs = []
ids = []
counts = []
labels = []
target_shape = (585, 153)

for r_id, row in tqdm(df.iterrows(), total = len(labels)):
    npy_path = das_to_numpy(row.DAS_File) 
    c = 0
    label = [0 for _ in range(8)]

    for j in range(8):
        if j != 4:
            vehicles = [v for v in eval(str(row[f"class_{j}_tracks"])) if v != 1]
            label[j] = min(len(vehicles), 5)
    c = sum(label)
    if c > 0:
        ids.append(r_id)
        imgs.append(np.load(npy_path)[:585, :153])
        counts.append(c)
        labels.append(label)
        

2334it [00:02, 1148.73it/s]


In [27]:
imgs = np.asarray(imgs)
ids = np.asarray(ids)
counts = np.asarray(counts)
labels = np.asarray(labels)

In [29]:
imgs = torch.from_numpy(imgs)
ids = torch.from_numpy(ids)
counts = torch.from_numpy(counts)
labels = torch.from_numpy(labels)

In [30]:
print(imgs.shape, ids.shape, counts.shape, labels.shape)

torch.Size([1391, 585, 153]) torch.Size([1391]) torch.Size([1391]) torch.Size([1391, 8])


In [33]:
torch.save({"imgs": imgs, "ids": ids, "counts": counts, "labels": labels}, "data.pt")

In [24]:
len(counts)

1391

In [25]:
labels

[[0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 2, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 3, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 2, 0, 0, 0, 0, 0],
 [0, 0, 2, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 2, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 4, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 2, 0, 0, 0, 0, 0],
 [0, 0, 2, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 2, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 2, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 

In [26]:
counts

[1,
 1,
 1,
 2,
 1,
 1,
 3,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 4,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 3,
 1,
 1,
 2,
 1,
 5,
 1,
 1,
 2,
 1,
 2,
 1,
 3,
 2,
 1,
 1,
 4,
 5,
 1,
 1,
 1,
 2,
 3,
 1,
 2,
 5,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 4,
 4,
 3,
 5,
 1,
 1,
 3,
 1,
 1,
 3,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 3,
 1,
 4,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 6,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, imgs, labels, transforms = lambda x: x):
        self.imgs = imgs
        self.labels = labels
        self.transforms = transforms
        self.noise = noise
    
    def __getitem__(self, i):
        return (self.transforms(self.imgs[i]), self.labels[i])
    
    def __len__(self):
        return len(self.imgs)